### Analyze Skills and Backgrounds of Data Scientists From 1 Million Data-Related Jobs

In [16]:
import requests
import pandas as pd
import json
from yarl import URL

In [11]:
!pip install pipe

In [15]:
!pip install yarl

     |████████████████████████████████| 271 kB 5.2 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 


In [43]:
!pip install observable_jupyter

In [ ]:
!pip install plotly

In [7]:
!pip install python-dotenv

In [28]:
from dotenv import load_dotenv
import os

load_dotenv()
TOKEN = os.getenv("DIFFBOT_TOKEN")

549a5fe69a72c50d28e7d36761986fb1


In [29]:
def download_file(query: str, size: int = 25, extract_data: bool = True):
    """Download data from Diffbot"""
    query_string = f"type=query&token={TOKEN}&query={query}&size={size}"
    url = URL.build(
        scheme="https",
        host="kg.diffbot.com",
        path="/kg/v3/dql",
        query_string=query_string,
    )
    r = requests.get(url)
    response = r.json()
    return response["data"] if extract_data else response

In [30]:

titles = [
    "data scientist",
    "data engineer",
    "data analyst",
    "machine learning engineer",
    "statistician",
    "data entry",
]

In [31]:
# Analyze education
from pipe import map, where


def get_top_major_per_position(title: str, num_top_majors: int = 10):
    data = download_file(
        f'type:Person employments.title:"{title}" facet:educations.major.name'
    )
    count = list(data | map(lambda res: res["count"]))[:num_top_majors]
    skills = list(data | map(lambda res: res["value"]))[:num_top_majors]
    return [{"name": name, "value": value} for name, value in zip(skills, count)]

In [32]:
majors_df = {title: pd.DataFrame(get_top_major_per_position(title)) for title in titles}

In [33]:
import pickle

pickle.dump(majors_df, open("majors_df.pkl", "wb"))

In [34]:
title_major = [
    {"name": title, "children": majors_df[title].to_dict("records")} for title in titles
]
title_major_all = {"name": "major", "children": title_major}

In [35]:
title_major_all["children"][0]

{'children': [{'name': 'computer science', 'value': 17465},
  {'name': 'business administration', 'value': 6960},
  {'name': 'mathematics', 'value': 6771},
  {'name': 'data science', 'value': 6481},
  {'name': 'statistics', 'value': 6107},
  {'name': 'physics', 'value': 5705},
  {'name': 'economics', 'value': 4402},
  {'name': 'mechanical engineering', 'value': 2966},
  {'name': 'computer engineering', 'value': 2693},
  {'name': 'information technology', 'value': 2634}],
 'name': 'data scientist'}

In [36]:
with open("majors.json", "w") as file:
    json.dump(title_major_all, file, indent=4)

In [37]:
import pickle

majors_df = pickle.load(open("majors_df.pkl", "rb"))
majors_df["data scientist"]

,name,value
0,computer science,17465
1,business administration,6960
2,mathematics,6771
3,data science,6481
4,statistics,6107
5,physics,5705
6,economics,4402
7,mechanical engineering,2966
8,computer engineering,2693
9,information technology,2634


In [38]:
import plotly.express as px


def plot_majors(title: str, majors_df: dict):
    majors = majors_df[title]
    return px.bar(data_frame=majors, x="name", y="value")

In [39]:
plot_majors('data scientist', majors_df)

In [40]:
plot_majors('data engineer', majors_df)

In [41]:
plot_majors('machine learning engineer', majors_df)

In [47]:
#Analyze Degree
def get_degree_count_per_position(title: str):
    data = download_file(
        f'type:Person employments.title:"{title}" facet:educations.degree.name', size=7
    )
    count = list(data | map(lambda res: res["count"]))
    degrees = list(data | map(lambda res: res["value"]))
    degrees_df = pd.DataFrame(
        [
            {"Title": title, "degree": name, "count": value}
            for name, value in zip(degrees, count)
        ]
    )
    return degrees_df[(degrees_df.degree != "-") & (degrees_df.degree != "diploma")]

In [48]:
degree_df = pd.concat(
    [get_degree_count_per_position(title) for title in titles]
).reset_index(drop=True)

In [49]:
degree_df.to_csv("raw_degrees.csv", index=False)

In [50]:
degree_df["Ratio to All Degrees"] = degree_df["count"] / max(degree_df["count"])

In [51]:
degree_df.degree = degree_df.degree.replace(
    {
        "bachelor's (4 year program)": "bachelor's",
        "master's (6 year program)": "master's",
        "phd or other doctorate": "phd",
        "associate's (2 year program)": "associate's",
        "high school or equivalent": "high school",
        "certificate/license": "certificate",
    }
)

In [52]:
degree_df.head(5)

,Title,degree,count,Ratio to All Degrees
0,data scientist,bachelor's,139220,0.310885
1,data scientist,master's,126404,0.282266
2,data scientist,phd,36243,0.080932
3,data scientist,high school,32513,0.072603
4,data scientist,certificate,9425,0.021046


In [53]:
degree_df.to_csv("degrees.csv", index=False)

In [54]:
#Analyze Gender

def get_gender_count_per_position(title: str):
    data = download_file(
        f'type:Person employments.title:"{title}" facet:gender.normalizedValue', size=2
    )
    count = list(data | map(lambda res: res["count"]))
    genders = list(data | map(lambda res: res["value"]))
    return pd.DataFrame(
        [
            {"Title": title, "gender": name, "count": value}
            for name, value in zip(genders, count)
        ]
    )

In [55]:
genders = [get_gender_count_per_position(title) for title in titles]
gender_df = pd.concat(genders)

In [56]:
gender_df.to_csv("genders.csv", index=False)

In [57]:
#Analyze Locations
import json

with open('states.json') as json_file:
    states_short = json.load(json_file)
states_short = {k.lower(): v for k, v in states_short.items()}

In [58]:
print(states_short)

{'baden-württemberg': 'BW', 'bavaria': 'BY', 'berlin': 'BE', 'brandenburg': 'BB', 'bremen': 'HB', 'hamburg': 'HH', 'hesse': 'HE', 'lower saxony': 'NI', 'mecklenburg-vorpommern': 'MV', 'north rhine-westphalia': 'NW', 'rhineland-palatinate': 'RP', 'saarland': 'SL', 'saxony': 'SN', 'saxony-anhalt': 'ST', 'schleswig-holstein': 'SH', 'thuringia': 'TH'}


In [59]:
def get_jobs_US_states(title: str):
    data = download_file(
        f'type:Person employments.title:"{title}" ' + 'facet:locations.{region.name country.name:"Germany"}', size=100
    )
    count = list(data | map(lambda res: res["count"]))
    states = list(data | map(lambda res: res["value"]))
    states = [state for state in states if state in states_short]
    return pd.DataFrame(
        [
            {"state": states_short[state], "count": value}
            for state, value in zip(states, count)
        ]
    )

In [60]:
state_jobs = get_jobs_US_states("data scientist")
state_jobs.to_csv("state_jobs.csv", index=False)

In [63]:
#Top 25 cities
def get_locations(title: str, groupby: str):
    data = download_file(f'type:Person employments.title:"{title}" {groupby}')
    count = list(data | map(lambda res: res["count"]))
    locations = list(data | map(lambda res: res["value"]))
    return pd.DataFrame(
        [{"location": name, "count": value} for name, value in zip(locations, count)]
    )


def plot_top_25_locations(title: str, locations: dict):
    data = locations[title]
    return px.bar(data_frame=data, x="location", y="count")

In [64]:
cities = {
    title: pd.DataFrame(get_locations(title, "facet:locations.city.name"))
    for title in titles
}

In [65]:
pickle.dump(cities, open("cities.pkl", "wb"))

In [66]:
plot_top_25_locations('data scientist', cities)

In [67]:
#Analyze Titles
def get_titles_per_position(title: str):
    data = download_file(
        f'type:Person employments.title:"{title}" facet:employments.title', size=200
    )

    relevant_titles = list(data | where(lambda res: title in res["value"]))
    count = list(relevant_titles | map(lambda res: res["count"]))
    titles = list(relevant_titles | map(lambda res: res["value"]))
    return [{"name": name, "value": value} for name, value in zip(titles, count)]


def plot_top_titles(title: str, top_titles: dict):
    data = top_titles[title]
    return px.bar(data_frame=data, x="name", y="value")

In [68]:
top_titles = {title: pd.DataFrame(get_titles_per_position(title)) for title in titles}
pickle.dump(top_titles, open("top_titles.pkl", "wb"))


In [77]:
top_titles["data scientist"]

,name,value
0,data scientist,117314
1,senior data scientist,18677
2,data scientist intern,7251
3,lead data scientist,5618
4,junior data scientist,4114
5,principal data scientist,2460
6,chief data scientist,2165
7,associate data scientist,2035
8,sr. data scientist,1377
9,data scientist ii,946


In [69]:
plot_top_titles('data scientist', top_titles)

In [70]:
title_types = [
    {"name": title, "children": top_titles[title].to_dict("records")}
    for title in titles
]

title_types_all = {"name": "title types", "children": title_types}

In [71]:
with open("titles.json", "w") as file:
    json.dump(title_types_all, file, indent=4)

In [72]:
#Count of Positions Over Time
from datetime import datetime


def get_num_position_per_year(title: str, year: int):
    query = (
        "type:Person employments.{"
        + f'title:"{title}"from<"{year}-12-31" or(to>"{year}-01-01", isCurrent:true)'
        + "}"
    )
    count = download_file(
        query,
        size=0,
        extract_data=False,
    )["hits"]
    return {
        "date": datetime(year=year, month=1, day=1),
        "name": title,
        "value": count,
    }

In [73]:
titles = [
    "data scientist",
    "data engineer",
    "data analyst",
    "machine learning engineer",
    "statistician",
    "data entry",
]
years = list(range(1994, 2022))

In [74]:
dates = [get_num_position_per_year(title, year) for title in titles for year in years]

In [75]:
dates_df = pd.DataFrame(dates)

In [76]:
dates_df.sample(10)

,date,name,value
118,2000-01-01,statistician,4931
134,2016-01-01,statistician,9227
94,2004-01-01,machine learning engineer,27
27,2021-01-01,data scientist,81601
41,2007-01-01,data engineer,6273
50,2016-01-01,data engineer,20411
154,2008-01-01,data entry,54562
13,2007-01-01,data scientist,1428
84,1994-01-01,machine learning engineer,13
23,2017-01-01,data scientist,52529


In [82]:
dates_df.to_csv("dates.csv")

In [83]:
px.line(dates_df, x="date", y="value", color="name")

In [84]:
#Count of Skills Over Time

def get_num_skill_per_year(title: str, year: int):
    query = (
        "type:Person employments.{"
        + f'title:"{title}"from<"{year}-12-31" or(to>"{year}-01-01", isCurrent:true)'
        + "} facet:skills.name"
    )
    data = download_file(query)
    count = list(data | map(lambda res: res["count"]))
    skills = list(data | map(lambda res: res["value"]))
    years = [datetime(year=year, month=1, day=1)] * len(data)
    skill_dict = list(zip(years, skills, count))
    return pd.DataFrame(skill_dict, columns=["date", "name", "value"])

In [85]:
dfs = [get_num_skill_per_year("data scientist", year) for year in years]

In [86]:
skills_df = pd.concat(dfs)

In [87]:
# Add the values of phython to the values of python
python_values = (
    skills_df[skills_df.name == "python"]
    .merge(skills_df[skills_df.name == "phython"], on="date", how="left")
    .fillna(0)
    .assign(total_value=lambda df_: df_.value_x + df_.value_y)["total_value"]
    .values
)

In [88]:
skills_df.loc[skills_df.name == "python", "value"] = python_values

In [89]:
skills_df = skills_df[skills_df.name != "phython"]

In [90]:
skills_df.head(10)

,date,name,value
0,1994-01-01,teaching,126.0
1,1994-01-01,economics,116.0
2,1994-01-01,management,95.0
3,1994-01-01,programming language,62.0
4,1994-01-01,mathematics,61.0
5,1994-01-01,software development,46.0
6,1994-01-01,data analysis,33.0
7,1994-01-01,machine learning,33.0
8,1994-01-01,project management,33.0
9,1994-01-01,chemistry,31.0


In [91]:
skills_df.to_csv("skills_over_time.csv")

In [92]:
px.line(skills_df, x="date", y="value", color="name")

In [93]:
def create_skill_df(job_title: str):
    data_f = download_file(
        f'type:Person employments.title:"{job_title.lower()}" facet:skills.name',
        size=100,
    )
    df = pd.DataFrame.from_dict(data_f).drop(columns=["callbackQuery"])
    df["Title"] = job_title
    df = df.rename(columns={"value": "skill"})
    return df


skills = [
    create_skill_df(title)
    for title in [
        "Data Scientist",
        "Data Engineer",
        "Data Analyst",
        "Machine Learning Engineer",
    ]
]

In [94]:
skill_count = pd.concat(skills)

In [95]:
skill_count.to_csv("all_skills.csv")

In [96]:
skill_count = pd.read_csv(
    "all_skills.csv",
    index_col=0,
)
skill_count.head(10)

,count,skill,Title
0,138442,teaching,Data Scientist
1,120458,programming language,Data Scientist
2,119969,economics,Data Scientist
3,110060,mathematics,Data Scientist
4,104907,software development,Data Scientist
5,83318,python,Data Scientist
6,82971,machine learning,Data Scientist
7,80848,robotics,Data Scientist
8,65168,data analysis,Data Scientist
9,56099,r,Data Scientist


In [100]:
topskill = skill_count[
    skill_count["skill"].isin(
        ["python", "data analysis", "sql", "machine learning", "statistics", "matlab", "deep learning", "mathematics"]
    )
]

topskill["Ratio to All Skills"] = topskill["count"].apply(
    lambda c: c / max(topskill["count"]))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [101]:
topskill[topskill["Title"] == "Data Analyst"]

,count,skill,Title,Ratio to All Skills
4,184780,data analysis,Data Analyst,1.000000
9,154031,mathematics,Data Analyst,0.833591
10,122840,sql,Data Analyst,0.664791
25,69860,python,Data Analyst,0.378071
34,55061,machine learning,Data Analyst,0.297981
45,40938,statistics,Data Analyst,0.221550
64,27759,matlab,Data Analyst,0.150227


In [102]:
topskill.to_csv("languages.csv")

In [79]:
!pip install pytrends

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=304c84c0cd881526c2c467a2e7d1be9aa2b5ce71b64b8daeabce5cc401b59710
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [80]:
#Analyze Trend in Germany

from pytrends.request import TrendReq


def plot_keyword_trend(title: str):
    pytrends = TrendReq(hl="en-US", tz=60)
    pytrends.build_payload(kw_list=[title])

    df = pytrends.interest_over_time()
    return px.line(data_frame=df, y=title)


In [81]:
plot_keyword_trend('data scientist')